In [ ]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import os
import adapt.utils.data_utils as prd
import adapt.loop as lp
import matplotlib.cm as cm
import adapt.ml.dl_subclass as dl
import tensorflow as tf
from keras.utils import to_categorical
import copy as cp

set_gpu()

In [ ]:

(x_train, y_train_lda), (x_test, y_test_lda) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0],-1)
y_train = to_categorical(y_train_lda)
x_test = x_test.reshape(x_test.shape[0],-1)
y_test = to_categorical(y_test_lda)

trainmlp = tf.data.Dataset.from_tensor_slices((x_train, y_train, y_train)).shuffle(x_train.shape[0],reshuffle_each_iteration=True).batch(128)

# return a new mnist dataset w/ pixels randomly permuted
def permute_mnist(mnist):
    perm_inds = list(range(mnist.shape[1]))
    np.random.shuffle(perm_inds)
    mnist2 = cp.deepcopy(mnist)
    mnist2 = np.transpose(np.array([mnist2[:,c] for c in perm_inds]))
    return mnist2

mnistb = permute_mnist(x_train)
trainmlp2 = tf.data.Dataset.from_tensor_slices((mnistb, y_train, y_train)).shuffle(x_train.shape[0],reshuffle_each_iteration=True).batch(128)


In [ ]:
ep =30
mlp, cnn, w, c = lp.train_models(trainmlp, trainmlp, x_train, y_train_lda, 10, ep=ep)
mlp2, cnn, w, c = lp.train_models(trainmlp, trainmlp2, x_train, y_train_lda, 10, ep=ep)

In [ ]:
test_ewc = dl.EWC(n_class=10)
test_ewc.call(x_train[:1,...])
test_ewc.star(mlp)
test_ewc.compute_diag_fim(x_train,y_train)

In [ ]:
ep = 100

lam = 50
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_loss2 = tf.keras.metrics.Mean(name='train_loss2')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

train_ewc = dl.get_train_ewc()

for epoch in range(ep):
    train_loss.reset_states()
    train_loss2.reset_states()
    train_accuracy.reset_states()
    for x, y, _ in trainmlp2:
        train_ewc(x, y, test_ewc, optimizer, train_loss, train_loss2, train_accuracy, lam=lam)
    if epoch == 0 or epoch == ep-1:
        print(f'Epoch {epoch + 1}, ', f'Loss: {train_loss.result():.2f}, ', f'Accuracy: {train_loss2.result() * 1:.2f} ')

In [ ]:
test_mod = dl.get_test()
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_mod(x_train, y_train, mlp, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(mnistb, y_train, mlp, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(x_train, y_train, mlp2, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(mnistb, y_train, mlp2, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(x_train, y_train, test_ewc, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(mnistb, y_train, test_ewc, test_loss, test_accuracy)
print(test_accuracy.result()*100)